In [10]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
import os
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import OpenAIEmbeddings
import requests
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

Ödev de kodların açıklanması istendiği için kodları neden yazdığımı ya da ne işe yaradıklarını yazıyorum

# Aşama 1: Veri Hazırlığı ve Ön İşleme

In [11]:
 #Dosyayı indir ve yükle
url = "https://raw.githubusercontent.com/respondcreate/nginx-access-log-frequency/master/example-access.log"
response = requests.get(url) #url den içeriği indir, requests kütüphanesi HTTP istekleri yapar
log_content = response.text # indirilen dosyanın textini metin olarak tuttum

# Log'u document olarak oluşturup grerkirse analiz ve işleme için...
docs = [Document(page_content=log_content)]

# benim kulandığım log dosyası küçük ve için de zaten az bilgi var bu yüzden herhangi bir veri silmiyorum ancak 
#eğer verim büyük olsaydı ve boş ya da kullanmamıza gerek olmayan çok veri olsaydı verileri düzenlerdim. "Data analysis" notebook'ta
#verileri pandas ile dataframe'e çevirip biraz düzenleme yaptım :).

In [12]:
# Metin içeriği yapmıştık onu parçalara ayırmalıyız
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50) # chunk_overlap değeri langchain sitesinde 200 olarak ayarlanmış...
splits = text_splitter.split_documents(docs) # docstaki metinleri belirlenen parçalara böler

# bunu yapmamızın sebebi textimizi ileride dil modeli için kullanacağız
# chunk_size metnin ne kadar büyük parçalara böleceğini belirliyo
# chunk_overlap bitişik parçaların bazı bölümlerinin tekrar edilmesini sağlar

### Vektörlere dönüştürüp, uygun bir vektör veri tabanına (örneğin, FAISS, Pinecone) yükleyin.

In [17]:
openai_api_key = "****"
os.environ["OPENAI_API_KEY"] = openai_api_key

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125") # 4o-latest daha yeni ve güncel olduğu için bunu kullanmıycam

llm = ChatOpenAI(model="chatgpt-4o-latest", max_tokens=20000, temperature=0.2)

# max_tokens = 20000 , modelin üretebileceği maksimum token saysını belirler. bizim modelimiz için fazla bile...
# temperature = 0.2 , değer küçüldükçe daha tutarlı cevaplar verir, değer büyüdükçe çeşitli cevaplar üretir.
# temperature değerini çok yükseltmek yaratıcı ve çeşitli değerler verse de gerçeklikten de uzaklaşabilir.

In [14]:
# Vektör deposu oluşturma (FAISS kullanarak)
embedding = OpenAIEmbeddings() # Word Embedding ile metinleri sayısal verilere dönüştürdük

vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
#yukarıda vektör deposu yaptım. FAISS yerel bir vektör deposu.

In [15]:
# Retriever oluşturma
retriever = vectorstore.as_retriever()

# Bu işlemi yapma sebebimiz FAISS vektör deposunu, veri arama ve alım işlemleri için kullanılabilir yapıyor 

# Aşama 2: RAG Modelinin Kurulumu

In [16]:
#modelin sorulara cevap verirken kullanacağı bir prompt şablonudur
prompt_template = """
Answer questions according to the following context:

{context}

Question: {question}
Answer:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"]) #  prompt oluşturmak için kullanılan araç

llm = OpenAI() #llm tanımlıyoruz, verilen prompta göre cevap üreticek

# RAG Chain oluşturma
# LLMChain Modeli ve promptu kullanarak sorguya uygun cevabı üreticek
rag_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

In [8]:
def ask_question(question):
    docs_context = retriever.get_relevant_documents(question) # soruyu temel alıp uluşturduğum vektör deposundan lazım olan belgeyi getirir
    context = "\n\n".join([doc.page_content for doc in docs_context])
    return rag_chain.run({"question": question, "context": context}) # rag_chainle yanıt üretiliir


## Genel Sorular

In [35]:
question = "En sık kullanılan IP adresi nedir"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 111.222.333.444


In [39]:
question = "En sık kullanılan IP adresi nedir ve kaç kez kullanılmıştır"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 
111.222.333.444 - 8 kez kullanılmıştır.


In [43]:
question = "HTTP isteklerinin çoğunluğu hangi durumu döndürmüştür?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: HTTP isteklerinin çoğunluğu 301 durum kodunu döndürmüştür.


In [47]:
question = "HTTP isteklerinin çoğunluğu hangi durumu , kaç kere döndürmüştür?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 
301 durum kodu, istekte bulunulan kaynağın kalıcı olarak başka bir URI'ye taşındığını ve bu yeni URI'nin kullanılması gerektiğini belirtir. Bu durum kodu, bir kaynağın URL'sinin değiştirilmesi veya bir web sitesinin taşınması gibi durumlarda sıkça kullanılır. Bu durumda, HTTP isteklerinin çoğu 301 durum kodunu 15 kez döndürmüştür.


In [83]:
question = "En sık erişilen endpoint nedir?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: /test-endpoint


In [100]:
question = "En çok kullanılan endpoint nedir?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: En çok kullanılan endpoint "/test-endpoint"tir. Bu endpoint 10 kez kullanılmıştır.


## Bazı Ayrıntı Sorular

In [128]:
question = "'libwww-perl/6.05' user agent'ı hangi IP adresi tarafından kullanılmıştır?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 33.444.55.666 IP adresi tarafından kullanılmıştır.


In [133]:
question = "Kaç tane 400 hata kodu döndüren istek vardır?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: There are 11 requests that returned a 400 error code.


In [146]:
question = "Kaç farklı user agent kullanılmış?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: There are at least 5 different user agents used in the given context. They are:
1. Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36
2. libwww-perl/6.05
3. Mozilla/3.0 (compatible; Indy Library)
4. -
5. NetcraftSurveyAgent/1.0; +info@netcraft.com)


## Zaman Bazlı Sorular

In [152]:
question = "En erken ve en geç yapılan istekler hangi saatlerde gerçekleşmiştir?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: En erken yapılan istek: [11/Mar/2016:05:09:15 -0500] "GET / HTTP/1.1" 301 184 "-" "libwww-perl/6.05" "-"
En geç yapılan istek: [11/Mar/2016:12:07:35 -0500] "GET / HTTP/1.1" 200 5454 "-" "-" "-"
Bu istekler arasındaki zaman aralığı: 7 saat 58 dakika 20 saniye


In [159]:
question = "Saat bazında isteklerin sayısında bir artış veya azalma eğilimi var mı?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: Evet, saat bazında isteklerin sayısında bir artış veya azalma eğilimi var. Örneğin, 11 Mart 2016 tarihinde 05:09:15 ile 05:09:22 arasında yapılan isteklerin sayısında bir artış görülürken, 05:47:53 saatinde yapılan tek bir istek görülmektedir. Ayrıca, 10:28:49 ile 10:28:54 saatleri arasında da bir artış görülmüştür. Bunun yanı sıra, 10:39:43 ve 10:39:44 saatlerinde de bir artış görülürken, 10:39:45 ile 10:39:47 saatleri arasında bir azalma eğilimi görülmektedir. 12:06:44 saatinde yapılan istek de öncesine göre daha azdır. Bu verilere göre saat bazında isteklerde bir artış veya azalma eğilimi olduğu söylenebilir.


In [160]:
question = "Dakika bazında isteklerin sayısında bir artış veya azalma eğilimi var mı?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 
Evet, dakika dakika isteklerin sayısında bir artış veya azalma eğilimi görülmektedir. Örneğin, 11 Mart 2016 tarihinde 10:28:49'da 4 adet istek yapılmışken, 10:28:50'de 1 adet istek yapılmıştır. Benzer şekilde, 10:39:43'te 1 adet istek yapılmışken, 10:39:45'te 3 adet istek yapılmıştır. Bu durum, dakika bazında isteklerin değişken bir şekilde artış veya azalış gösterdiğini göstermektedir.


## Hata Bazlı Sorular

In [168]:
question = "sadece 404 hatası alan istekler var mı?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 
Hayır, listede 404 hatası alan isteklerin olmadığı görülmektedir.


In [179]:
question = "400 hatası alan kaç istek vardır?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 
11


## Güvenlik ve Anomali Tespiti

In [202]:
question = "Bir IP adresi belirli bir zaman diliminde ardışık olarak çok sayıda istek göndermiş mi?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: Evet, 111.222.333.444 IP adresi 11 Mart 2016 tarihinde ardışık zaman dilimlerinde birçok istek göndermiştir. Örneğin, 10:39:43 ve 10:39:45 arasında 5 farklı istek gönderilmiştir. Ayrıca, 10:39:53 ve 10:40:18 arasında da 2 farklı istek gönderilmiştir. Toplamda 8 farklı istek gönderilmiştir.


In [203]:
question = "'GET /admin' isteğini gönderen IP adresleri hangileridir?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: 
888.99.11.22
999.111.222.333


In [204]:
question = "'libwww-perl/6.05' gibi belirli bir user agent'la yapılan isteklerde bir anormallik var mı?"
answer = ask_question(question)

print("Cevap:", answer)

Cevap: Evet, bu user agent ile yapılan isteklerde bir anormallik var gibi görünüyor. Öncelikle, bu user agent'a sahip olan IP adresinin yaptığı isteklerde sürekli olarak aynı URL'e yapılan isteklerde bir artış görülmekte ve bu isteklerin neredeyse tamamı 400 (Bad Request) kodu ile sonuçlanmaktadır. Bu durum, muhtemelen bir bot veya spam saldırısı gibi görünmektedir ve dikkat edilmesi gereken bir durum olabilir. Ayrıca, bu user agent ile yapılan isteklerin bir kısmı da boş bir istek (GET / HTTP/1.1) şeklinde gözükmektedir, bu da yine isteklerin anormal bir şekilde yapıldığını göstermektedir.
